In [64]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import time
import requests
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

In [65]:
df = pd.read_csv('./pa_fires_in_ph.csv')

The majority of many columns have a -4 value, which indicates the information was hidden for privacy reasons, which occurs when the number of residents is fewer than 10.

In [66]:
df.replace(-4, np.nan, inplace=True)

In [67]:
pd.set_option('display.max_rows', 200)
(df.isna().sum() / len(df)).sort_values(ascending=False)

necta_nm                      1.000000
dpv                           1.000000
dpvact                        1.000000
dpvnost                       1.000000
place_inc2kx                  1.000000
urb_out                       1.000000
dpvrc                         1.000000
zip_class                     0.998279
c1psrc                        0.977631
apt_no                        0.966220
apt_type                      0.966129
c1pdrc                        0.958794
annl_expns_amnt_prev_yr       0.945209
pct_asian                     0.945028
pct_native_american           0.945028
pct_black                     0.945028
pct_minority                  0.945028
pct_age85plus                 0.945028
pct_age62plus                 0.945028
pct_disabled_lt62             0.945028
pct_age51_61                  0.945028
pct_age25_50                  0.945028
pct_lt24_head                 0.945028
pct_disabled_all              0.945028
pct_disabled_ge62             0.945028
months_waiting           

In [68]:
# Assuming your DataFrame is named 'df'
columns_to_drop = [
    'x', 'y', 'objectid', 'participant_code', 'std_addr', 'apt_no', 'rc2kx', 'stm2kx',
    'dev_buil_nu_entrance', 'ha_phn_num', 'ha_fax_num', 'ha_email_addr_text',
    'exec_dir_phone', 'exec_dir_fax', 'exec_dir_email',
    'lat', 'lon',
    'c1pgrc', 'c1pprb', 'c1pdrc', 'c1psrc', 'c1parc', 'c1pzrc',
    'project_name', 'building_name', 'building_number',
    'zcta2kx', 'dpbc', 'dpbc_cksum', 'std_zip11',
    'national_bldg_id', 'curcnty_nm', 'curcnty', 'curcosub', 'curcosub_nm',
    'place2kx', 'place_nm2kx',
    'msa', 'cbsa', 'necta', 'necta_nm', 'last_updt_dttm', 'development_code',
    'state2kx', 'cnty_nm2kx','cnty2kx','tract2kx','bg2kx','block2kx', 'cbsa_nm',
    'fcd_fips91','hlc', 'std_city', 'std_st',
    'county_level'	,'place_level',	'tract_level',	'blkgrp_level',
    'place_cc2kx', 'place_inc2kx'
]

df.drop(columns=columns_to_drop, inplace=True)

In [69]:
df['construction_year'] = pd.to_datetime(df['construct_date']).dt.year
df['dofa_actual_year'] = pd.to_datetime(df['dofa_actual_dt']).dt.year
df = df.drop(['construct_date', 'dofa_actual_dt'], axis=1)

In [70]:
df.sample()

,formal_participant_name,building_type_code,building_status_type_code,total_dwelling_units,acc_units,total_occupied,regular_vacant,total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,spending_per_month_prev_yr,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,dpvact,dpvnost,msa_nm,metro,micro,dpv,dpvrc,urb_out,std_zip5,std_zip9,zip_class,addr_type,apt_type,msgusps,lvl2kx,ur,msg2kx,rad_chap_indr,rad_type,annl_expns_amnt,annl_expns_amnt_prev_yr,pha_total_units,house_fires,cooking_fires,all_fires,house_fires_20-21,cooking_fires_20-21,all_fires_20-21,construction_year,dofa_actual_year
2365,Housing Authority of the City of Pittsburgh,ES,INAPCP,61,60,59,1,61,96.720001,59,100.0,7.0,5.08,1.0,59.0,355.0,2356.0,1983.0,15552.0,15552.0,0.0,1.69,54.240002,27.120001,16.950001,1.69,0.0,98.309998,17.1,98.309998,74.580002,0.0,0.0,81.360001,0.0,100.0,39.66,40.68,0.0,1.69,0.0,98.309998,20.34,89.830002,89.830002,0.0,0.0,0.0,0.0,92.0,100.0,48.0,100.0,0.0,0.0,0.0,71.279999,6.04,61.700001,0.0,98.309998,1.69,100.0,14375.0,NaN,NaN,"Pittsburgh, PA MSA",1.0,0.0,NaN,NaN,NaN,15224.0,1055.0,NaN,H,NaN,90% Overall probable correctness,R,U,** Street-Level Rooftop **,N,,1668034.0,1427787.0,3154,0,0,0,0,0,0,2004.0,2004.0


In [71]:
# Assuming your dataframe is named df
placeholder = "MISSING_VALUE"  # or any other string you'd like to use as placeholder

# List of columns to be one-hot encoded
columns_to_encode = [
     'msa_nm', 'building_type_code', 
    'building_status_type_code', 'formal_participant_name',
    'addr_type', 'apt_type', 'zip_class', 'lvl2kx', 'ur', 'msg2kx', 
    'rad_chap_indr', 'rad_type'
]

# Fill NaN values with placeholder
for column in columns_to_encode:
    df[column] = df[column].fillna(placeholder)

# One-hot encode the columns
for column in columns_to_encode:
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df.drop([column], axis=1, inplace=True)

In [72]:
from sklearn.preprocessing import LabelEncoder

df.msgusps = LabelEncoder().fit_transform(df.msgusps)

In [73]:
df.sample()

,total_dwelling_units,acc_units,total_occupied,regular_vacant,total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,spending_per_month_prev_yr,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,dpvact,dpvnost,metro,micro,dpv,dpvrc,urb_out,std_zip5,std_zip9,msgusps,annl_expns_amnt,annl_expns_amnt_prev_yr,pha_total_units,house_fires,cooking_fires,all_fires,house_fires_20-21,cooking_fires_20-21,all_fires_20-21,construction_year,dofa_actual_year,"msa_nm_Allentown-Bethlehem-Easton, PA MSA","msa_nm_Altoona, PA MSA","msa_nm_Erie, PA MSA","msa_nm_Harrisburg-Lebanon-Carlisle, PA MSA","msa_nm_Johnstown, PA MSA","msa_nm_Lancaster, PA MSA",msa_nm_MISSING_VALUE,"msa_nm_Philadelphia, PA-NJ PMSA","msa_nm_Pittsburgh, PA MSA","msa_nm_Reading, PA MSA","msa_nm_Scranton--Wilkes-Barre--Hazleton, PA MSA","msa_nm_Sharon, PA MSA","msa_nm_Williamsport, PA MSA","msa_nm_York, PA MSA",building_type_code_ES,building_type_code_RW,building_type_code_SD,building_type_code_SF,building_type_code_WU,building_status_type_code_DDAPRD,building_status_type_code_DDDRFT,building_status_type_code_DDPROP,building_status_type_code_INAPCP,building_status_type_code_RMIPRP,formal_participant_name_ALLEGHENY COUNTY HOUSING AUTHORITY,formal_participant_name_ALTOONA HOUSING AUTHORITY,formal_participant_name_Allentown Housing Authority,formal_participant_name_Bethlehem Housing Authority,formal_participant_name_Bradford County Housing Authority,formal_participant_name_Bucks County Housing Authority,formal_participant_name_CLEARFIELD COUNTY HOUSING AUTHORITY,formal_participant_name_CONNELLSVILLE HOUSING AUTHORITY,formal_participant_name_Carbon County Housing Authority,formal_participant_name_Chester Housing Authority,formal_participant_name_Columbia County Housing Authority,formal_participant_name_Easton Housing Authority,formal_participant_name_FAYETTE COUNTY HOUSING AUTHORITY,formal_participant_name_HOUSING AUTH CO OF LAWRENCE,formal_participant_name_HOUSING AUTHORITY OF INDIANA COUNTY,formal_participant_name_HOUSING AUTHORITY OF THE CITY OF DUBOIS,formal_participant_name_HOUSING AUTHORITY OF THE CITY OF ERIE,formal_participant_name_HOUSING AUTHORITY OF THE CITY OF FRANKLIN,formal_participant_name_HOUSING AUTHORITY OF THE CITY OF MCKEESPORT,formal_participant_name_HOUSING AUTHORITY OF THE CITY OF MEADVILLE,formal_participant_name_HOUSING AUTHORITY OF THE CITY OF OIL CITY,formal_participant_name_HOUSING AUTHORITY OF THE COUNTY OF BEAVER,formal_participant_name_HOUSING AUTHORITY OF THE COUNTY OF BEDFORD,formal_participant_name_HOUSING AUTHORITY OF THE COUNTY OF BLAIR,formal_participant_name_HOUSING AUTHORITY OF THE COUNTY OF ELK,formal_participant_name_HOUSING AUTHORITY OF THE COUNTY OF ERIE,formal_participant_name_HOUSING AUTHORITY OF THE COUNTY OF GREENE,formal_participant_name_HOUSING AUTHORITY OF THE COUNTY OF HUNTINGDON,formal_participant_name_HOUSING AUTHORITY OF THE COUNTY OF JEFFERSON,formal_participant_name_HOUSING AUTHORITY OF THE COUNTY OF MCKEAN,formal_participant_name_HOUSING AUTHORITY OF THE COUNTY OF WARREN,formal_participant_name_Harrisburg Housing Authority,formal_participant_name_Housing Auth of the County of Northumberland,formal_participant_name_Housing Authority County of Delaware,formal_participant_name_Housing Authority of the City of Pittsburgh,formal_participant_name_Housing Autho

In [76]:
import pandas as pd
import numpy as np
from scipy import stats

# Sample dataframe
# df = pd.DataFrame({ ... })

# Compute the Spearman correlation
correlations = df.corrwith(df['house_fires'], method='spearman').drop('house_fires')

# Compute the significance
n = len(df)
# The t-statistic formula remains the same since the Spearman rank correlation is distributed in a manner similar to the Pearson correlation under the null hypothesis.
t_stats = correlations * np.sqrt((n-2) / (1-correlations**2))
p_values = 2*(1 - stats.t.cdf(np.abs(t_stats), df=n-2))  # two-tailed

# Create results dataframe
results_df = pd.DataFrame({
    'Correlation': correlations,
    'p-value': p_values
})

# Filter out columns with insignificant correlations (e.g., p-value > 0.05)
significant_results = results_df[results_df['p-value'] < 0.05]

significant_results.sort_values('Correlation')

,Correlation,p-value
people_per_unit,-0.214615,0.000000e+00
chldrn_mbr_cnt,-0.212386,0.000000e+00
pct_female_head_child,-0.211974,0.000000e+00
pct_1adult,-0.208162,0.000000e+00
pct_overhoused,-0.205147,0.000000e+00
pct_age25_50,-0.200318,0.000000e+00
pct_female_head,-0.191260,0.000000e+00
pct_bed2,-0.184268,0.000000e+00
pct_welfare_major,-0.169022,0.000000e+00
pct_bed3,-0.160264,0.000000e+00


In [75]:
df.to_csv('pa_fires_in_ph_clean.csv', index=False)